In [ ]:
# # Tried it, didn't worked, move to something else
# # pb avec orateurs, etc.
# # mais semble confirmer les 242 paragraphes

# import pandas as pd

# # Chemin vers le fichier XML
# xml_path = "CRSANR5L16S2024O1N220.xml"

# # Lire tous les paragraphes comme lignes du DataFrame
# df_paragraphs = pd.read_xml(
#     xml_path,
#     xpath=".//ns:paragraphe",
#     namespaces={"ns": "http://schemas.assemblee-nationale.fr/referentiel"},
#     encoding="utf-8"
# )

# print(df_paragraphs.shape)

In [ ]:
# V1

# import xml.etree.ElementTree as ET
# import csv

# # Charger le fichier
# tree = ET.parse("CRSANR5L16S2024O1N220.xml")
# root = tree.getroot()
# ns = {"ns": "http://schemas.assemblee-nationale.fr/referentiel"}

# # Métadonnées globales
# uid = root.find(".//ns:uid", ns).text
# date_seance = root.find(".//ns:dateSeanceJour", ns).text
# president_elem = root.find(".//ns:presidentSeance", ns)
# president = president_elem.text if president_elem is not None else ""

# # Préparer lignes
# rows = []

# # Parcourir les <point> (chaque point = sujet)
# for point in root.findall(".//ns:point", ns):
#     sujet = point.find("ns:texte", ns)
#     sujet_texte = sujet.text.strip() if sujet is not None and sujet.text else ""
#     valeur_ptsodj = point.attrib.get("valeur_ptsodj", "")

#     # Parcourir les <paragraphe> de chaque point
#     for para in point.findall("ns:paragraphe", ns):
#         texte_elem = para.find("ns:texte", ns)
#         if texte_elem is None:
#             continue

#         # Texte brut et attribut stime
#         texte = "".join(texte_elem.itertext()).strip()
#         stime = texte_elem.attrib.get("stime", "")

#         # Orateur
#         orateur_elem = para.find(".//ns:orateur", ns)
#         nom, qualite, id_orateur = "", "", ""
#         if orateur_elem is not None:
#             nom = (
#                 orateur_elem.find("ns:nom", ns).text
#                 if orateur_elem.find("ns:nom", ns) is not None
#                 else ""
#             )
#             qualite = (
#                 orateur_elem.find("ns:qualite", ns).text
#                 if orateur_elem.find("ns:qualite", ns) is not None
#                 else ""
#             )
#             id_orateur = (
#                 orateur_elem.find("ns:id", ns).text
#                 if orateur_elem.find("ns:id", ns) is not None
#                 else ""
#             )

#         # Attributs de paragraphe
#         row = {
#             "UID": uid,
#             "Date séance": date_seance,
#             "Président": president,
#             "Sujet": sujet_texte,
#             "Valeur ODJ": valeur_ptsodj,
#             "ID paragraphe": para.attrib.get("id_syceron", ""),
#             "Ordre séance": para.attrib.get("ordre_absolu_seance", ""),
#             "Code grammaire": para.attrib.get("code_grammaire", ""),
#             "Code style": para.attrib.get("code_style", ""),
#             "Code parole": para.attrib.get("code_parole", ""),
#             "Role débat": para.attrib.get("roledebat", ""),
#             "Nom orateur": nom,
#             "Qualité orateur": qualite,
#             "ID orateur": id_orateur,
#             "Texte": texte,
#             "stime": stime,
#         }
#         rows.append(row)

# # Écrire en CSV
# with open("assemblee_debat_complet.csv", "w", newline="", encoding="utf-8") as f:
#     writer = csv.DictWriter(f, fieldnames=list(rows[0].keys()))
#     writer.writeheader()
#     writer.writerows(rows)

# print("Fichier CSV généré : assemblee_debat_complet.csv")


In [ ]:
# ON MANQUAIT DES ÉLÉMENTS

# # V2 : récupérer plus de metadonnées pour le contexte
# # ça ça marche-ish (visiblement on rate quelques éléments ?)

# import xml.etree.ElementTree as ET
# import csv

# # Charger le fichier XML
# tree = ET.parse("CRSANR5L16S2024O1N220.xml")
# root = tree.getroot()
# ns = {"ns": "http://schemas.assemblee-nationale.fr/referentiel"}


# # Métadonnées générales du document
# def get_text(path):
#     elem = root.find(path, ns)
#     return elem.text.strip() if elem is not None and elem.text else ""


# metadata = {
#     "UID": get_text(".//ns:uid"),
#     "SeanceRef": get_text(".//ns:seanceRef"),
#     "SessionRef": get_text(".//ns:sessionRef"),
#     "DateSeance": get_text(".//ns:dateSeance"),
#     "DateSeanceJour": get_text(".//ns:dateSeanceJour"),
#     "NumSeanceJour": get_text(".//ns:numSeanceJour"),
#     "NumSeance": get_text(".//ns:numSeance"),
#     "TypeAssemblee": get_text(".//ns:typeAssemblee"),
#     "Legislature": get_text(".//ns:legislature"),
#     "Session": get_text(".//ns:session"),
#     "NomFichierJO": get_text(".//ns:nomFichierJo"),
#     # "Validite": get_text(".//ns:validite"),
#     # "Etat": get_text(".//ns:etat"),
#     # "Diffusion": get_text(".//ns:diffusion"),
#     # "Version": get_text(".//ns:version"),
#     # "Environnement": get_text(".//ns:environnement"),
#     "President": get_text(".//ns:presidentSeance"),
# }

# # Extraire chaque paragraphe comme une ligne
# rows = []

# for point in root.findall(".//ns:point", ns):
#     sujet = point.find("ns:texte", ns)
#     sujet_texte = sujet.text.strip() if sujet is not None and sujet.text else ""
#     valeur_ptsodj = point.attrib.get("valeur_ptsodj", "")

#     for para in point.findall("ns:paragraphe", ns):
#         texte_elem = para.find("ns:texte", ns)
#         if texte_elem is None:
#             continue

#         texte = "".join(texte_elem.itertext()).strip()
#         stime = texte_elem.attrib.get("stime", "")

#         orateur_elem = para.find(".//ns:orateur", ns)
#         nom, qualite, id_orateur = "", "", ""
#         if orateur_elem is not None:
#             nom = (
#                 orateur_elem.find("ns:nom", ns).text
#                 if orateur_elem.find("ns:nom", ns) is not None
#                 else ""
#             )
#             qualite = (
#                 orateur_elem.find("ns:qualite", ns).text
#                 if orateur_elem.find("ns:qualite", ns) is not None
#                 else ""
#             )
#             id_orateur = (
#                 orateur_elem.find("ns:id", ns).text
#                 if orateur_elem.find("ns:id", ns) is not None
#                 else ""
#             )

#         # Attributs de paragraphe
#         row = {
#             **metadata,
#             "Sujet": sujet_texte,
#             "Valeur_ODJ": valeur_ptsodj,
#             "ID_paragraphe": para.attrib.get("id_syceron", ""),
#             "Ordre_seance": para.attrib.get("ordre_absolu_seance", ""),
#             "Code_grammaire": para.attrib.get("code_grammaire", ""),
#             "Code_style": para.attrib.get("code_style", ""),
#             "Code_parole": para.attrib.get("code_parole", ""),
#             "Role_debat": para.attrib.get("roledebat", ""),
#             "Nom_orateur": nom,
#             "Qualite_orateur": qualite,
#             "ID_orateur": id_orateur,
#             "stime": stime,
#             "Texte": texte,
#         }

#         rows.append(row)

# # Écriture CSV
# output_file = "V2.csv"
# with open(output_file, "w", newline="", encoding="utf-8") as f:
#     fieldnames = list(rows[0].keys())
#     writer = csv.DictWriter(f, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
#     writer.writeheader()
#     writer.writerows(rows)